# Data-driven persona creation Q4 2020
Unlike the other iterations, this iteration will cluster the persona on Q4 2020 data and later on classify the Q1 2021 data into the cluster created. This iteration will also use kprototype instead of kmeans, to use both categorical and numeric data.

In [1]:
# Remove the warnings for presentation of the notebook. During the development, the warnings were not ignored.
import warnings
warnings.filterwarnings('ignore')

First, import numpy, pandas, and datetime libraries, read the Q4 dummy dataset, and drop the columns not needed.

In [2]:
import pandas as pd
import numpy as np
import datetime as dt

df = pd.read_csv('Q4-all.csv')
df = df.drop(columns=['year', 'month'])
df.head()

,Unnamed: 0,userID,companyID,country,timestamp,quarter,weekNumber,dayNumber,activity,page
0,0,34,5,Indonesia,2020-10-04 23:02:44,4,40,4,Page,https://en.wikipedia.org/wiki/Financial_services
1,1,145,44,Turkey,2020-10-22 19:26:33,4,43,22,Page,https://en.wikipedia.org/wiki/Financial_techno...
2,2,195,8,Turkey,2020-12-31 09:02:33,4,53,31,Event,https://en.wikipedia.org/wiki/Bookkeeping
3,3,16,7,Turkey,2020-11-17 16:32:24,4,47,17,Page,https://en.wikipedia.org/wiki/Main_Page
4,4,65,42,Turkey,2020-12-02 01:32:00,4,49,2,Event,https://en.wikipedia.org/wiki/Financial_services


Then, we assign a new column called event consisting of the type of event the user did on a certain time and location. This algorithm applied only to rows which activity value equals to event. The rows which acitivity value equals to Page will be assigned a 0. Event 1 represent upload invoice, 2 represent download invoice and 3 represent share invoice.

In [3]:
from random import seed
from random import randint

seed(1)

getEvent = df[df['activity'] == 'Event']

events = []

index = len(getEvent.index)

for _ in range(index):
    event = randint(1, 3)
    events.append(event)

getEvent['event'] = events
df = df.merge(getEvent[['Unnamed: 0','event']], on='Unnamed: 0', how='left')
df['event'] = df['event'].fillna(0)
df.head()

,Unnamed: 0,userID,companyID,country,timestamp,quarter,weekNumber,dayNumber,activity,page,event
0,0,34,5,Indonesia,2020-10-04 23:02:44,4,40,4,Page,https://en.wikipedia.org/wiki/Financial_services,0.0
1,1,145,44,Turkey,2020-10-22 19:26:33,4,43,22,Page,https://en.wikipedia.org/wiki/Financial_techno...,0.0
2,2,195,8,Turkey,2020-12-31 09:02:33,4,53,31,Event,https://en.wikipedia.org/wiki/Bookkeeping,1.0
3,3,16,7,Turkey,2020-11-17 16:32:24,4,47,17,Page,https://en.wikipedia.org/wiki/Main_Page,0.0
4,4,65,42,Turkey,2020-12-02 01:32:00,4,49,2,Event,https://en.wikipedia.org/wiki/Financial_services,3.0


The timestamp is first updated into the local timestamp, since the timestamp is local European time where the analytic dashboard is set up.

In [4]:
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

In [5]:
import datetime as dt
import pytz

localIndTime = df[['country', 'timestamp']]
localIndTime = df[df['country'] == 'Indonesia']
localIndTime['visitTimeLocal'] = pd.to_datetime(localIndTime['timestamp'])
localIndTime['visitTimeLocal'] = localIndTime['visitTimeLocal'].dt.tz_localize('UTC').dt.tz_convert('Asia/Bangkok')
mapping5 = dict(localIndTime[['timestamp', 'visitTimeLocal']].values)
df['visitTimeLocal'] = df[df['country'] == 'Indonesia'].timestamp.map(mapping5)
df.head()

,Unnamed: 0,userID,companyID,country,timestamp,quarter,weekNumber,dayNumber,activity,page,event,visitTimeLocal
0,0,34,5,Indonesia,2020-10-04 23:02:44,4,40,4,Page,https://en.wikipedia.org/wiki/Financial_services,0.0,2020-10-05 06:02:44+07:00
1,1,145,44,Turkey,2020-10-22 19:26:33,4,43,22,Page,https://en.wikipedia.org/wiki/Financial_techno...,0.0,NaT
2,2,195,8,Turkey,2020-12-31 09:02:33,4,53,31,Event,https://en.wikipedia.org/wiki/Bookkeeping,1.0,NaT
3,3,16,7,Turkey,2020-11-17 16:32:24,4,47,17,Page,https://en.wikipedia.org/wiki/Main_Page,0.0,NaT
4,4,65,42,Turkey,2020-12-02 01:32:00,4,49,2,Event,https://en.wikipedia.org/wiki/Financial_services,3.0,NaT


In [6]:
localTurTime = df[['country', 'timestamp']]
localTurTime = df[df['country'] == 'Turkey']
localTurTime['visitTimeLocal1'] = pd.to_datetime(localTurTime['timestamp'])
localTurTime['visitTimeLocal1'] = localTurTime['visitTimeLocal1'].dt.tz_localize('UTC').dt.tz_convert('Europe/Istanbul')
mapping6 = dict(localTurTime[['timestamp', 'visitTimeLocal1']].values)
df['visitTimeLocal1'] = df[df['country'] == 'Turkey'].timestamp.map(mapping6)
df.head()

,Unnamed: 0,userID,companyID,country,timestamp,quarter,weekNumber,dayNumber,activity,page,event,visitTimeLocal,visitTimeLocal1
0,0,34,5,Indonesia,2020-10-04 23:02:44,4,40,4,Page,https://en.wikipedia.org/wiki/Financial_services,0.0,2020-10-05 06:02:44+07:00,NaT
1,1,145,44,Turkey,2020-10-22 19:26:33,4,43,22,Page,https://en.wikipedia.org/wiki/Financial_techno...,0.0,NaT,2020-10-22 22:26:33+03:00
2,2,195,8,Turkey,2020-12-31 09:02:33,4,53,31,Event,https://en.wikipedia.org/wiki/Bookkeeping,1.0,NaT,2020-12-31 12:02:33+03:00
3,3,16,7,Turkey,2020-11-17 16:32:24,4,47,17,Page,https://en.wikipedia.org/wiki/Main_Page,0.0,NaT,2020-11-17 19:32:24+03:00
4,4,65,42,Turkey,2020-12-02 01:32:00,4,49,2,Event,https://en.wikipedia.org/wiki/Financial_services,3.0,NaT,2020-12-02 04:32:00+03:00


In [7]:
df['visitTimeLocal'] = df['visitTimeLocal'].apply(lambda t: t.replace(tzinfo=None))
df['visitTimeLocal1'] = df['visitTimeLocal1'].apply(lambda t: t.replace(tzinfo=None))
df['timestamp_local'] = df['visitTimeLocal'].combine_first(df['visitTimeLocal1'])
df['timestamp_local'] = df['visitTimeLocal'].fillna(df['visitTimeLocal1'])
df.head()

,Unnamed: 0,userID,companyID,country,timestamp,quarter,weekNumber,dayNumber,activity,page,event,visitTimeLocal,visitTimeLocal1,timestamp_local
0,0,34,5,Indonesia,2020-10-04 23:02:44,4,40,4,Page,https://en.wikipedia.org/wiki/Financial_services,0.0,2020-10-05 06:02:44,NaT,2020-10-05 06:02:44
1,1,145,44,Turkey,2020-10-22 19:26:33,4,43,22,Page,https://en.wikipedia.org/wiki/Financial_techno...,0.0,NaT,2020-10-22 22:26:33,2020-10-22 22:26:33
2,2,195,8,Turkey,2020-12-31 09:02:33,4,53,31,Event,https://en.wikipedia.org/wiki/Bookkeeping,1.0,NaT,2020-12-31 12:02:33,2020-12-31 12:02:33
3,3,16,7,Turkey,2020-11-17 16:32:24,4,47,17,Page,https://en.wikipedia.org/wiki/Main_Page,0.0,NaT,2020-11-17 19:32:24,2020-11-17 19:32:24
4,4,65,42,Turkey,2020-12-02 01:32:00,4,49,2,Event,https://en.wikipedia.org/wiki/Financial_services,3.0,NaT,2020-12-02 04:32:00,2020-12-02 04:32:00


Then, the hour, day-of-week and week number in the quarter information from the local timestamp is extracted in the below cell.

In [8]:
df['hour'] = df['timestamp_local'].dt.hour
df['day'] = df['timestamp_local'].dt.dayofweek
df.head()

,Unnamed: 0,userID,companyID,country,timestamp,quarter,weekNumber,dayNumber,activity,page,event,visitTimeLocal,visitTimeLocal1,timestamp_local,hour,day
0,0,34,5,Indonesia,2020-10-04 23:02:44,4,40,4,Page,https://en.wikipedia.org/wiki/Financial_services,0.0,2020-10-05 06:02:44,NaT,2020-10-05 06:02:44,6,0
1,1,145,44,Turkey,2020-10-22 19:26:33,4,43,22,Page,https://en.wikipedia.org/wiki/Financial_techno...,0.0,NaT,2020-10-22 22:26:33,2020-10-22 22:26:33,22,3
2,2,195,8,Turkey,2020-12-31 09:02:33,4,53,31,Event,https://en.wikipedia.org/wiki/Bookkeeping,1.0,NaT,2020-12-31 12:02:33,2020-12-31 12:02:33,12,3
3,3,16,7,Turkey,2020-11-17 16:32:24,4,47,17,Page,https://en.wikipedia.org/wiki/Main_Page,0.0,NaT,2020-11-17 19:32:24,2020-11-17 19:32:24,19,1
4,4,65,42,Turkey,2020-12-02 01:32:00,4,49,2,Event,https://en.wikipedia.org/wiki/Financial_services,3.0,NaT,2020-12-02 04:32:00,2020-12-02 04:32:00,4,2


In [9]:
week = df[['weekNumber']]
week = week.sort_values(by=['weekNumber']).reset_index()
week = week.reset_index()
week = week['weekNumber'].value_counts().reset_index()
week = week.sort_values(by=['index'])
week.rename(columns={'index': 'weekNumber', 'weekNumber': 'count'}, inplace=True)
week = week.reset_index()
week['week'] = week.index + 1

df = df.merge(week, on='weekNumber', how='left')
df = df.drop(columns=['index', 'count'])
df.head()

,Unnamed: 0,userID,companyID,country,timestamp,quarter,weekNumber,dayNumber,activity,page,event,visitTimeLocal,visitTimeLocal1,timestamp_local,hour,day,week
0,0,34,5,Indonesia,2020-10-04 23:02:44,4,40,4,Page,https://en.wikipedia.org/wiki/Financial_services,0.0,2020-10-05 06:02:44,NaT,2020-10-05 06:02:44,6,0,1
1,1,145,44,Turkey,2020-10-22 19:26:33,4,43,22,Page,https://en.wikipedia.org/wiki/Financial_techno...,0.0,NaT,2020-10-22 22:26:33,2020-10-22 22:26:33,22,3,4
2,2,195,8,Turkey,2020-12-31 09:02:33,4,53,31,Event,https://en.wikipedia.org/wiki/Bookkeeping,1.0,NaT,2020-12-31 12:02:33,2020-12-31 12:02:33,12,3,14
3,3,16,7,Turkey,2020-11-17 16:32:24,4,47,17,Page,https://en.wikipedia.org/wiki/Main_Page,0.0,NaT,2020-11-17 19:32:24,2020-11-17 19:32:24,19,1,8
4,4,65,42,Turkey,2020-12-02 01:32:00,4,49,2,Event,https://en.wikipedia.org/wiki/Financial_services,3.0,NaT,2020-12-02 04:32:00,2020-12-02 04:32:00,4,2,10


As per the focus group particiapnts' request, the user count per company should be added back into the calculation per below.

In [10]:
userCountPerCompany = df[['companyID', 'userID']]
userCountPerCompany['companySize'] = userCountPerCompany.groupby(by='companyID')['userID'].transform('count')
mapping = dict(userCountPerCompany[['companyID', 'companySize']].values)
df['userCountPerCompany'] = df.companyID.map(mapping)
df.head()

,Unnamed: 0,userID,companyID,country,timestamp,quarter,weekNumber,dayNumber,activity,page,event,visitTimeLocal,visitTimeLocal1,timestamp_local,hour,day,week,userCountPerCompany
0,0,34,5,Indonesia,2020-10-04 23:02:44,4,40,4,Page,https://en.wikipedia.org/wiki/Financial_services,0.0,2020-10-05 06:02:44,NaT,2020-10-05 06:02:44,6,0,1,977
1,1,145,44,Turkey,2020-10-22 19:26:33,4,43,22,Page,https://en.wikipedia.org/wiki/Financial_techno...,0.0,NaT,2020-10-22 22:26:33,2020-10-22 22:26:33,22,3,4,2424
2,2,195,8,Turkey,2020-12-31 09:02:33,4,53,31,Event,https://en.wikipedia.org/wiki/Bookkeeping,1.0,NaT,2020-12-31 12:02:33,2020-12-31 12:02:33,12,3,14,989
3,3,16,7,Turkey,2020-11-17 16:32:24,4,47,17,Page,https://en.wikipedia.org/wiki/Main_Page,0.0,NaT,2020-11-17 19:32:24,2020-11-17 19:32:24,19,1,8,2419
4,4,65,42,Turkey,2020-12-02 01:32:00,4,49,2,Event,https://en.wikipedia.org/wiki/Financial_services,3.0,NaT,2020-12-02 04:32:00,2020-12-02 04:32:00,4,2,10,2012


Then, subsetting the dataframe before clustering with Kprototype.

In [11]:
df1 = df[['userID', 'companyID', 'country', 'activity', 'event', 'page', 'timestamp', 'hour', 'day', 'week', 'userCountPerCompany']]
df1.head()

,userID,companyID,country,activity,event,page,timestamp,hour,day,week,userCountPerCompany
0,34,5,Indonesia,Page,0.0,https://en.wikipedia.org/wiki/Financial_services,2020-10-04 23:02:44,6,0,1,977
1,145,44,Turkey,Page,0.0,https://en.wikipedia.org/wiki/Financial_techno...,2020-10-22 19:26:33,22,3,4,2424
2,195,8,Turkey,Event,1.0,https://en.wikipedia.org/wiki/Bookkeeping,2020-12-31 09:02:33,12,3,14,989
3,16,7,Turkey,Page,0.0,https://en.wikipedia.org/wiki/Main_Page,2020-11-17 16:32:24,19,1,8,2419
4,65,42,Turkey,Event,3.0,https://en.wikipedia.org/wiki/Financial_services,2020-12-02 01:32:00,4,2,10,2012


First, assign the dataframe values into mark array and inform the array which columns are with numeric value.

In [12]:
mark_array=df1.values

In [13]:
mark_array[:, 0] = mark_array[:, 0].astype(float)
mark_array[:, 1] = mark_array[:, 1].astype(float)
mark_array[:, 4] = mark_array[:, 4].astype(float)
mark_array[:, 7] = mark_array[:, 7].astype(float)
mark_array[:, 8] = mark_array[:, 8].astype(float)
mark_array[:, 9] = mark_array[:, 9].astype(float)
mark_array[:, 10] = mark_array[:, 10].astype(float)

mark_array

array([[34.0, 5.0, 'Indonesia', ..., 0.0, 1.0, 977.0],
       [145.0, 44.0, 'Turkey', ..., 3.0, 4.0, 2424.0],
       [195.0, 8.0, 'Turkey', ..., 3.0, 14.0, 989.0],
       ...,
       [189.0, 13.0, 'Indonesia', ..., 3.0, 14.0, 2594.0],
       [128.0, 39.0, 'Turkey', ..., 2.0, 12.0, 3432.0],
       [50.0, 15.0, 'Turkey', ..., 0.0, 4.0, 1960.0]], dtype=object)

Now, assigning the dataset to 4 clusters with 20 max iteration to minimize the algorithm running time. Then, fit the mark array and categorical variables into kprototype model, print the centroids and put back the cluster value into the dataframe in the column cluster.

In [14]:
from kmodes.kprototypes import KPrototypes

kproto = KPrototypes(n_clusters=4, verbose=2,max_iter=20)
clusters = kproto.fit_predict(mark_array, categorical=[2, 3, 5, 6])

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/20, moves: 0, ncost: 11243097835.322697
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 2, iteration: 1/20, moves: 0, ncost: 10709650810.863972
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 3, iteration: 1/20, moves: 12775, ncost: 31324753899.04722
Run: 3, iteration: 2/20, moves: 13843, ncost: 26487127549.547916
Run: 3, iteration: 3/20, moves: 23752, ncost: 18808653261.101715
Run: 3, iteration: 4/20, moves: 85, ncost: 18808485925.80373
Run: 3, iteration: 5/20, moves: 0, ncost: 18808485925.80373
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 4, iteration: 1/20, moves: 6845, ncost: 10686048282.078844
Run: 4, iteration: 2/20, moves: 0, ncost: 10686048282.078844
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 5, iteration: 1/20, moves: 28791,

In [15]:
print(kproto.cluster_centroids_)

[[95.95401383357786 23.30423391322574 0.9917836931460909
  11.475937958499266 3.009012785579543 7.5109620624607 1638.4129532592747
  'Turkey' 'Page' 'https://en.wikipedia.org/wiki/Accounting'
  Timestamp('2020-10-02 07:12:36')]
 [93.519595448799 36.78767383059419 0.9917825537294563 11.54203539823009
  2.9878002528445005 7.500948166877371 3178.387610619469 'Turkey' 'Page'
  'https://en.wikipedia.org/wiki/Main_Page'
  Timestamp('2020-10-04 18:51:54')]
 [92.7509976057462 49.0 0.9938148443735035 11.558659217877095
  3.0139664804469275 7.410015961691939 5012.0 'Turkey' 'Page'
  'https://en.wikipedia.org/wiki/Accounting'
  Timestamp('2020-10-11 06:37:41')]
 [110.01201602136182 26.37535761968339 0.9958039290482548
  11.53436327802149 3.004768262445165 7.443448407400344 2430.487443575561
  'Turkey' 'Page' 'https://en.wikipedia.org/wiki/Financial_services'
  Timestamp('2020-10-02 12:29:35')]]


In [16]:
cluster_dict=[]
for c in clusters:
    cluster_dict.append(c)

In [17]:
cluster_dict

[0,
 3,
 0,
 3,
 0,
 3,
 2,
 3,
 0,
 0,
 3,
 3,
 0,
 0,
 3,
 0,
 1,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 2,
 3,
 1,
 0,
 0,
 0,
 2,
 3,
 0,
 3,
 3,
 0,
 1,
 3,
 3,
 0,
 1,
 3,
 0,
 3,
 2,
 0,
 3,
 0,
 3,
 1,
 3,
 3,
 0,
 3,
 0,
 0,
 3,
 3,
 3,
 0,
 0,
 3,
 3,
 3,
 1,
 1,
 3,
 1,
 1,
 3,
 3,
 0,
 0,
 1,
 0,
 0,
 0,
 3,
 3,
 0,
 0,
 0,
 3,
 1,
 3,
 0,
 3,
 0,
 3,
 1,
 3,
 3,
 1,
 3,
 0,
 0,
 3,
 3,
 1,
 0,
 2,
 0,
 1,
 3,
 3,
 0,
 1,
 0,
 0,
 0,
 3,
 0,
 3,
 3,
 0,
 3,
 0,
 3,
 3,
 1,
 2,
 1,
 0,
 0,
 0,
 1,
 3,
 3,
 2,
 0,
 1,
 0,
 0,
 0,
 3,
 1,
 3,
 0,
 1,
 0,
 3,
 3,
 2,
 0,
 3,
 2,
 3,
 3,
 2,
 3,
 1,
 3,
 3,
 0,
 0,
 3,
 0,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 3,
 3,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 1,
 1,
 3,
 0,
 1,
 3,
 3,
 3,
 0,
 1,
 3,
 3,
 3,
 0,
 0,
 0,
 1,
 2,
 3,
 0,
 3,
 0,
 0,
 3,
 0,
 1,
 0,
 1,
 0,
 3,
 3,
 0,
 2,
 0,
 1,
 0,
 0,
 3,
 0,
 0,
 0,
 3,
 1,
 2,
 0,
 3,
 0,
 3,
 3,
 0,
 2,
 0,


In [18]:
df1['cluster']=cluster_dict
df1.head()

,userID,companyID,country,activity,event,page,timestamp,hour,day,week,userCountPerCompany,cluster
0,34,5,Indonesia,Page,0.0,https://en.wikipedia.org/wiki/Financial_services,2020-10-04 23:02:44,6,0,1,977,0
1,145,44,Turkey,Page,0.0,https://en.wikipedia.org/wiki/Financial_techno...,2020-10-22 19:26:33,22,3,4,2424,3
2,195,8,Turkey,Event,1.0,https://en.wikipedia.org/wiki/Bookkeeping,2020-12-31 09:02:33,12,3,14,989,0
3,16,7,Turkey,Page,0.0,https://en.wikipedia.org/wiki/Main_Page,2020-11-17 16:32:24,19,1,8,2419,3
4,65,42,Turkey,Event,3.0,https://en.wikipedia.org/wiki/Financial_services,2020-12-02 01:32:00,4,2,10,2012,0


Let's count the amount of each cluster to see the magnitude and then transfer them to a csv file to be used in further process.

In [21]:
df1['cluster'].value_counts()

0    47710
3    31458
1    15820
2     5012
Name: cluster, dtype: int64

In [22]:
df1.to_csv('final-Q4.csv')